In [1]:
# a setting for my cluster; ignore it
import os
os.environ['CUDA_MPS_PIPE_DIRECTORY'] = "/tmp/nvidia-mps"
os.environ['CUDA_MPS_LOG_DIRECTORY'] = "/tmp/nvidia-log"

In [2]:
%%time
from nngp import *

CPU times: user 1.25 s, sys: 210 ms, total: 1.46 s
Wall time: 2.37 s


In [3]:
random_state = 4623457
ndim = 10
N_train, N_test, N_pool = 200, 200, 1000
layers = [128,64,32]

np.random.seed(random_state)

In [4]:
from scipy.optimize import rosen
def ans(x):
    return rosen(x.T)[:,None]
X_train = np.random.random((N_train, ndim))
y_train = ans(X_train)

X_pool = np.random.random((N_pool, ndim))
y_pool = ans(X_pool)

X_test = np.random.random((N_test, ndim))
y_test = ans(X_test)

print('shapes:', X_train.shape, y_train.shape)
print('shapes:', X_test.shape, y_test.shape)
print('shapes:', X_pool.shape, y_pool.shape)

shapes: (200, 10) (200, 1)
shapes: (200, 10) (200, 1)
shapes: (1000, 10) (1000, 1)


In [5]:
tf.reset_default_graph()
nn = NN(
    ndim = ndim,
    random_state = random_state,
    layers = layers
)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [6]:
try:
    sess.close()
except:
    pass
# a setting for my cluster; ignore it
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# global init
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session(config=config)
sess.run(init)

In [7]:
nn.train(
    session = sess,
    X_train = X_train,
    y_train = y_train,
    X_test = X_test,
    y_test = y_test,
    X_val = X_test,
    y_val = y_test,
)

rmses = [np.sqrt(mse(nn.predict(sess, data = X_test), y_test))]

for al_iteration in range(1, 10):
    print(f'[{al_iteration}] BEFORE:')
    print('shapes:', X_train.shape, y_train.shape)
    print('shapes:', X_test.shape, y_test.shape)
    print('shapes:', X_pool.shape, y_pool.shape)
    
    gpue = nn.get_nngp_UE(sess, X_train, y_train, X_pool, y_pool)
    
    X_train, y_train, X_pool, y_pool = \
        update_learning_sets(X_train,
                             y_train,
                             X_pool,
                             y_pool,
                             gpue,
                             sample_size = 100)
    
    print(f'[{al_iteration}] AFTER:')
    print('shapes:', X_train.shape, y_train.shape)
    print('shapes:', X_test.shape, y_test.shape)
    print('shapes:', X_pool.shape, y_pool.shape)
    
    nn.train(
        session = sess,
        X_train = X_train,
        y_train = y_train,
        X_test = X_test,
        y_test = y_test,
        X_val = X_test,
        y_val = y_test,
    )
    rmses.append(np.sqrt(mse(nn.predict(sess, data = X_test),
            y_test)))

[100] RMSE train:188.330 test:193.352 val:193.352 patience:3
[200] RMSE train:185.401 test:190.416 val:190.416 patience:3
[300] RMSE train:173.383 test:178.338 val:178.338 patience:3
[400] RMSE train:148.431 test:153.247 val:153.247 patience:3
[500] RMSE train:111.355 test:116.008 val:116.008 patience:3
[600] RMSE train:77.711 test:82.224 val:82.224 patience:3
[700] RMSE train:65.722 test:69.886 val:69.886 patience:3
[800] RMSE train:64.474 test:68.452 val:68.452 patience:3
[900] RMSE train:64.307 test:68.304 val:68.304 patience:3
[1000] RMSE train:64.168 test:68.237 val:68.237 patience:3
[1100] RMSE train:64.023 test:68.174 val:68.174 patience:3
[1200] RMSE train:63.871 test:68.111 val:68.111 patience:3
[1300] RMSE train:63.713 test:68.047 val:68.047 patience:3
[1400] RMSE train:63.550 test:67.983 val:67.983 patience:3
[1500] RMSE train:63.382 test:67.918 val:67.918 patience:3
[1600] RMSE train:63.210 test:67.854 val:67.854 patience:3
[1700] RMSE train:63.035 test:67.790 val:67.790 pa

In [8]:
rmses

[67.25156403814879,
 28.85082313310023,
 26.9729508550653,
 23.72526908062341,
 23.15615076282674,
 22.094892234408363,
 20.22693135272602,
 19.764543329123125,
 19.515263433354697,
 19.575532301736178]

In [9]:
nn.predict(sess,
           data = X_test[:3])

array([[201.61533],
       [190.08832],
       [267.92703]], dtype=float32)

In [10]:
y_test[:3]

array([[183.97308059],
       [186.273626  ],
       [310.47204144]])

In [11]:
gpue = nn.get_nngp_UE(sess, X_train, y_train, X_pool, y_pool)
mcd_ue = nn.get_mcd_UE(sess, X_pool)
gpue, mcd_ue

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([241.29450282, 178.62767736, 176.70902005, 116.13721032,
        149.69919447, 168.44082626, 274.13486521, 189.43445536,
        175.77393818, 193.04612476, 285.7587523 , 176.24976246,
        120.43920411, 252.25571197, 164.48160033,  49.28101007,
        191.21251995, 276.40485406, 259.26177001, 154.27310926,
         87.08936413, 144.23034991, 125.6479077 , 151.55738739,
        145.41873832, 157.08860935, 293.99861717, 226.8078781 ,
        173.20591558, 287.37230461, 294.14128571, 125.97612937,
        212.02765846, 159.18223435, 11

In [13]:
print('shapes:', X_train.shape, y_train.shape)
print('shapes:', X_test.shape, y_test.shape)
print('shapes:', X_pool.shape, y_pool.shape)

shapes: (1100, 10) (1100, 1)
shapes: (200, 10) (200, 1)
shapes: (100, 10) (100, 1)


In [14]:
X_train, y_train, X_pool, y_pool = \
    update_learning_sets(X_train,
                         y_train,
                         X_pool,
                         y_pool,
                         gpue,
                         sample_size = 10)

In [15]:
print('shapes:', X_train.shape, y_train.shape)
print('shapes:', X_test.shape, y_test.shape)
print('shapes:', X_pool.shape, y_pool.shape)

shapes: (1110, 10) (1110, 1)
shapes: (200, 10) (200, 1)
shapes: (90, 10) (90, 1)
